In [1]:
import torch
from transformers import pipeline
from auto_gptq import AutoGPTQForCausalLM

from utils.prompter import Prompter

MODEL = "j5ng/kullm-12.8b-GPTQ-8bit"
model = AutoGPTQForCausalLM.from_quantized(MODEL, device="cuda:1", use_triton=False)

pipe = pipeline('text-generation', model=model,tokenizer=MODEL)

prompter = Prompter("kullm")

/home/ubuntu/miniconda3/envs/torch/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
GPTNeoXGPTQForCausalLM hasn't fused attention module yet, will skip inject fused attention.
GPTNeoXGPTQForCausalLM hasn't fused mlp module yet, will skip inject fused mlp.
Xformers is not installed correctly. If you want to use memory_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.
The model 'GPTNeoXGPTQForCausalLM' is not supported for text-generation. Supported models are ['BartForCausalLM', 'BertLMHeadModel', 'BertGenerationDecoder', 'BigBirdForCausalLM', 'BigBirdPegasusForCausalLM', 'BioGptForCausalLM', 'BlenderbotForCausalLM', 'BlenderbotSmallForCausalLM', 'BloomForCausalLM', 'CamembertForCausalLM', 'CodeGenForCausalLM', 'CpmAntForCausalLM', 'CTRLLMHeadMo

In [2]:
def infer(instruction="", input_text=""):
    prompt = prompter.generate_prompt(instruction, input_text)
    output = pipe(
        prompt, max_length=512,
        temperature=0.2,
        repetition_penalty=3.0,
        num_beams=5,
        eos_token_id=2
    )
    s = output[0]["generated_text"]
    result = prompter.get_response(s)

    return result

In [4]:
instruction = """
백종원은 1966년 충남 예산군에서 집안의 종손으로 태어났다. 중학교 시절 상경해 강남 8학군 서울고등학교[23]를 졸업하고 연세대학교에 입학했다. 고등학교 졸업 직후엔 잠시 서울특별시 장한평에 위치한 중고차 시장에서 자동차 중개업자로 활동하기도 했다.
2019년, KBS의 토크쇼 프로그램인 대화의 희열에 출연하여 어렸을 적 이야기를 많이 했는데, 만석꾼이었던 증조할아버지 백영기(白榮基)의 피를 이어받은 영향인지 어렸을 때부터 장사꾼 기질이 있다고 스스로 자각하고 있었다고 한다.
9살 때에는 산에 놀러갔다가 본 버섯 농장에서 별다른 투자도 안 한 거 같은데 돈이 된다는 이야기를 듣고 꿈을 버섯 농사로 정한 적도 있었고, 초등학교 4학년때는 캔이 아닌 병에 음료가 나올 시절에 음료수 병을 보고 '저게 돈이 될 것 같다'고 생각해 학교 리어카를 빌려 오락 시간과 보물찾기 같은 시간을 다 건너뛰고 리어카 6개 분량의 공병을 모아서 고물상에 갖다 팔아 큰 돈을 벌었다고 한다.
그리고 5학년 1학기까지 이렇게 돈을 벌었고, 방위성금으로 다 냈다고 한다. 
"""
result = infer(instruction=instruction, input_text="백종원의 증조할아버지는 누구?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


백종원의 증조할아버지는 백영기(白榮基)입니다.


In [5]:
result = infer(input_text="백종원의 증조할아버지는 누구?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


백종원의 증조할아버지는 백종수입니다.


In [5]:
instruction = """
손흥민(한국 한자: 孫興慜, 1992년 7월 8일 ~ )은 대한민국의 축구 선수로 현재 잉글랜드 프리미어리그 토트넘 홋스퍼에서 윙어로 활약하고 있다.
또한 대한민국 축구 국가대표팀의 주장이자 2018년 아시안 게임 금메달리스트이며 영국에서는 애칭인 "쏘니"(Sonny)로 불린다.
아시아 선수로서는 역대 최초로 프리미어리그 공식 베스트 일레븐과 아시아 선수 최초의 프리미어리그 득점왕은 물론 FIFA 푸스카스상까지 휩쓸었고 2022년에는 축구 선수로는 최초로 체육훈장 청룡장 수훈자가 되었다.
손흥민은 현재 리그 100호를 넣어서 화제가 되고 있다.
"""
result = infer(instruction=instruction, input_text="손흥민의 애칭은 뭐야?")
print(result)

result = infer(instruction=instruction, input_text="손흥민의 포지션은 어디야?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


손흥민의 애칭은 "쏘니"입니다.
손흥민의 포지션은 윙어입니다.


In [6]:
result = infer(input_text="손흥민의 애칭은 뭐야?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


"손세이셔날"은 손흥민의 애칭입니다.


In [7]:
instruction = """
이번 전세사기 특별법에는 조세 채권 안분을 비롯, 전세 사기 피해자에 우선매수권 부여, LH공사 공공임대 활용 등의 내용이 포함됐다.
전세사기로 피해를 입은 세입자로 분류되면 살고 있는 집이 경매로 넘어갔을 경우 우선매수권을 부여받고,
경매로 집을 낙찰받을 경우 금융지원을 받을 수 있다.
"""
result = infer(instruction=instruction, input_text="전세사기 특별법에는 어떤 내용이 포함되었나요?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


전세사기 특별법에는 다음과 같은 내용이 포함되어 있습니다:

1. 조세 채권 안분: 전세 사기 피해자에게 우선매수권을 부여하고, LH공사 공공임대를 활용하여 금융 지원을 받을 수 있도록 하였습니다.

2. 우선매수권: 전세 사기 피해자로 분류된 세입자는 살고 있는 집이 경매로 넘어갔을 경우 우선매수권을 갖게 됩니다. 이를 통해 안정적인 주거 공간을 확보할 수 있습니다.

3. 금융 지원: 전세 사기 피해자로 분류된 세입자는 경매로 집을 낙찰받을 경우 금융 지원을 받을 수 있습니다. 이를 통해 경제적 부담을 줄일 수 있습니다.


In [8]:
result = infer(input_text="전세사기 특별법에는 어떤 내용이 포함되었나요?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


전세사기 특별법은 미국에서 전세 계약 사기를 방지하고 피해자를 구제하기 위해 제정된 법입니다. 이 법은 다음과 같은 몇 가지 중요한 조항을 포함하고 있습니다:

1. 세입자의 보증금 보호: 세입자가 임대인에게 보증금을 청구할 수 있는 권리를 보장합니다.

2. 임대인의 의무: 임대인은 세입자의 보증금을 안전하게 보관해야 할 의무가 있으며, 이를 위반할 경우 형사 처벌을 받을 수 있습니다.

3. 증거 수집: 임대인은 세입자가 보증금을 돌려받지 못했다는 사실을 증명할 수 있는 증거를 확보해야 합니다.

4. 민사 소송: 세입자는 임대인을 상대로 민사 소송을 제기하여 보증금을 반환받을 수 있습니다.

5. 벌금 및 징역형: 임대인이 세입자의 보증금을 돌려주지 않거나 허위 정보를 제공하는 등의 행위로 인해 유죄 판결을 받으면 최대 5만 달러의 벌금 또는 3년 이하의 징역형에 처해질 수 있습니다.

6. 대중의 인식: 이 법은 전세 계약 사기 사건에 대한 대중의 인식을 높이고 피해를 예방하는 데 도움이 되었습니다.


In [9]:
instruction = """
국방정책실 고공단 가급(1급) 일반직 또는 장성급 장교.
통상 부처들에선 장, 차관 뒤를 잇는 서열 3위로 인식되는 게 기조실장인데, 국방부에선 정책실장이다.
[18] 기조실장의 보좌를 받는 차관을 패스하고 직접 장관과 소통하며 밀접하게 지내는 자리이다.
예비역 장성이 장관으로 임명되다 보니 정책실장도 장관과 같은 군종 출신 예비역이 임명[19]되고, 장관 교체 시 정책실장도 사임하는 것이 관행이다.
통상적으로 정책실 소속 부서를 거쳐간 장교는 대령 혹은 장군으로 진급하는 경우가 많고, 실장으로는 예비역 육군 중장이 보직하는 게 일반적이었다.
문재인 정부 이후엔 비육군, 준장, 소장 등이 보직하는 경우가 많아졌지만, 예나 지금이나 예비역 장성이 오는 곳임에는 변함이 없다.
"""
result = infer(instruction=instruction, input_text="국방정책실에는 반드시 예비역 장성들만 올 수있나요?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


국방정책실에서 근무하는 사람들은 반드시 예비역 장성들이어야 하는 것은 아닙니다. 그러나 이러한 직위는 통상적으로 예비역 장성들이 맡아왔습니다. 문재인 정부 이후에는 비육군, 준장, 소장 등이 정책실에서 근무할 수 있게 되었습니다. 하지만 여전히 예비역 장성들이 정책실에서 근무하는 것이 일반적입니다.


In [10]:
result = infer(input_text="국방정책실에는 반드시 예비역 장성들만 올 수있나요?")
print(result)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


예, 국방정책실(DPA)에는 반드시 예비역 장성들만 참석할 수 있습니다. DPA는 미국 국방부의 정책 및 전략적 계획을 수립하고 실행하는 역할을 담당하기 때문에 군 경력이 있는 사람들로 구성된 것이 이상적입니다.
